<a href="https://colab.research.google.com/github/EricRoh-kr/st9_youtubevid/blob/main/st9_youtubevid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# 필요한 라이브러리 설치
import subprocess
import sys
import os
import zipfile
import threading
import shutil
from datetime import datetime
today = datetime.now().strftime("%m%d")

# Google Colab 환경 확인
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    print("💻 로컬 환경에서 실행 중입니다.")

def install_ytdlp():
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "yt-dlp"])
    except subprocess.CalledProcessError:
        print("❌ yt-dlp 설치에 실패했습니다.")
        return False
    return True

def clear_existing_folder(folder_name):
    if os.path.exists(folder_name):
        try:
            for filename in os.listdir(folder_name):
                file_path = os.path.join(folder_name, filename)
                if os.path.isfile(file_path):
                    os.remove(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            return True
        except Exception as e:
            print(f"❌ 폴더 정리 중 오류 발생: {str(e)}")
            return False
    return True

def get_video_title(url):
    """유튜브 URL에서 영상 제목만 가져오기"""
    try:
        result = subprocess.run(
            ['yt-dlp', '--get-title', url],
            capture_output=True, text=True
        )
        if result.returncode == 0:
            return result.stdout.strip()
        else:
            return None
    except Exception:
        return None


def fetch_and_print_title(url, index):
    """스레드로 제목 가져와서 출력"""
    title = get_video_title(url)
    if title:
        print(f"\n🎵 [{index}] 제목: {title}")
    else:
        print(f"\n⚠️ [{index}] 제목 확인 실패")


def get_quality_choice():
    """화질 선택"""
    print("\n" + "="*40)
    print("🎯 화질 선택")
    print("="*40)
    print("1. 📱 일반화질 (720p 이하, 빠른 다운로드)")
    print("2. 🎬 고화질 (최고 품질, 용량 큼)")
    print("-"*40)

    while True:
        choice = input("선택하세요 (1 또는 2): ").strip()
        if choice == '1':
            print("✅ 일반화질 모드가 선택되었습니다.")
            return 'normal'
        elif choice == '2':
            print("✅ 고화질 모드가 선택되었습니다.")
            return 'high'
        else:
            print("❌ 1 또는 2를 입력해주세요!")

def create_download_folder():
    """다운로드 폴더 생성"""
    folder_name = "영상저장함"

    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"📁 폴더 생성: {folder_name}")
    else:
        print(f"📁 기존 폴더 사용: {folder_name}")

    return folder_name

def create_zip_file(folder_name):
    """폴더를 ZIP으로만 압축 (자동 다운로드 없음)"""
    zip_filename = f"{today}_유튜브다운로드.zip"
    if os.path.exists(zip_filename):
        os.remove(zip_filename)

    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED, compresslevel=1) as zipf:
        for file in os.listdir(folder_name):
            file_path = os.path.join(folder_name, file)
            if os.path.isfile(file_path):
                zipf.write(file_path, file)
    print(f"📦 ZIP 파일 생성 완료: {zip_filename}")
    print("💡 Colab 왼쪽 파일탐색기에서 직접 다운로드하세요.")

def get_video_urls():
    """링크만 입력 → 제목은 나중에 한번에 출력"""
    urls = []
    print("\n" + "="*80)
    print("🔗 유튜브 링크 입력")
    print("="*80)
    print("1️⃣ 다운로드할 유튜브 링크를 입력해주세요.")
    print("2️⃣ 링크를 하나씩 입력하고 엔터를 누르세요.")
    print("3️⃣ 모든 링크를 입력했으면 '다운로드'를 입력하세요.")
    print("-"*80)

    while True:
        try:
            print(f"\n🔗 링크 #{len(urls)+1} 입력:")
            url_input = input("   │ ").strip()
            if url_input.lower() == '다운로드':
                if urls:
                    print(f"\n✅ 총 {len(urls)}개 링크가 입력되었습니다.")
                    break
                else:
                    print("❌ 최소 하나의 링크를 입력해주세요!")
                    continue
            if url_input:
                if 'youtube.com' in url_input or 'youtu.be' in url_input:
                    urls.append(url_input)
                    print(f"✅ 추가됨: {len(urls)}번째 영상")
                else:
                    print("❌ 올바른 유튜브 링크를 입력해주세요!")
            else:
                print("💡 링크를 입력하거나 '다운로드'를 입력해주세요.")
        except KeyboardInterrupt:
            print("\n\n⏹️  입력이 중단되었습니다.")
            return []

    # 제목 확인은 입력이 끝난 뒤
    print("\n🎥 영상 확인 중...")
    for i, url in enumerate(urls, 1):
        title = get_video_title(url)
        if title:
            print(f"[{i}] {title}")
        else:
            print(f"[{i}] 제목 확인 실패")
    return urls


def create_zip_file(folder_name):
    zip_filename = f"{today}_유튜브다운로드.zip"
    if os.path.exists(zip_filename):
        os.remove(zip_filename)
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED, compresslevel=1) as zipf:
        for file in os.listdir(folder_name):
            file_path = os.path.join(folder_name, file)
            if os.path.isfile(file_path):
                zipf.write(file_path, file)
    print(f"📦 ZIP 파일 생성 완료: {zip_filename}")
    return zip_filename   # ← 추가


def download_videos(urls, quality, folder_name):
    """비디오 다운로드"""
    print(f"\n🚀 다운로드 시작! (총 {len(urls)}개 영상)")
    print("="*50)

    # 화질 설정
    if quality == 'high':
        format_option = "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best"
        print("🎬 고화질 모드로 다운로드합니다.")
    else:
        format_option = "best[height<=720]/best"
        print("🎬 일반화질 모드로 다운로드합니다.")

    success_count = 0
    failed_urls = []

  # 수정된 다운로드 루프
    for i, url in enumerate(urls, 1):
        print(f"\n{i}/{len(urls)} 다운로드 중...")

        try:
            cmd = [
                "yt-dlp",
                "-f", format_option,
                "-o", f"{folder_name}/%(title)s.%(ext)s",
                "--no-warnings",
                url
            ]
            result = subprocess.run(cmd, capture_output=True, text=True, encoding="utf-8")

            if result.returncode == 0:
                print(f"✅ {i}번째 영상 다운로드 완료!")
                success_count += 1
            else:
                print(f"❌ {i}번째 영상 다운로드 실패!")
                print(f"   오류 원인: {result.stderr.strip()}")
                failed_urls.append(url)

        except Exception as e:
            print(f"❌ {i}번째 영상 처리 중 예외 발생: {e}")
            failed_urls.append(url)   # ← except 안으로 넣어야 함


    print("\n" + "="*50)
    print("📊 다운로드 완료!")
    print("="*50)
    print(f"✅ 성공: {success_count}개")
    print(f"❌ 실패: {len(failed_urls)}개")

    if failed_urls:
        print("\n❌ 실패한 링크들:")
        for url in failed_urls:
            print(f"   - {url}")

    if IN_COLAB:
        print(f"\n📁 Colab 내 위치: {os.path.abspath(folder_name)}")
    else:
        print(f"\n📁 다운로드 위치: {os.path.abspath(folder_name)}")

    return success_count > 0


def remove_zip(zip_filename):
    """ZIP 파일 삭제"""
    if os.path.exists(zip_filename):
        try:
            os.remove(zip_filename)
            print(f"🗑 ZIP 파일 삭제 완료: {zip_filename}")
        except Exception as e:
            print(f"❌ ZIP 파일 삭제 실패: {e}")

def main():
    """메인 함수"""
    try:
        subprocess.run(['yt-dlp', '--version'], check=True, capture_output=True)
    except (subprocess.CalledProcessError, FileNotFoundError):
        if not install_ytdlp():
            return

    try:
        urls = get_video_urls()
        quality = get_quality_choice()
        folder_name = create_download_folder()
        has_downloads = download_videos(urls, quality, folder_name)

        if has_downloads and IN_COLAB:
          print("\n" + "="*50)
          print("📦 압축 및 다운로드 준비")
          print("="*50)
          zip_filename = create_zip_file(folder_name)
          files.download(zip_filename)
          clear_existing_folder(folder_name)
          print("⬇️ ZIP 파일 자동 다운로드 시작")

        elif has_downloads:
            print("💻 로컬 환경: 파일들이 폴더에 저장되었습니다.")

        if not IN_COLAB:
            input("\n아무 키나 눌러서 종료하세요...")

    except KeyboardInterrupt:
        print("\n\n⏹️  사용자에 의해 중단되었습니다.")
    except Exception as e:
        print(f"\n❌ 오류가 발생했습니다: {str(e)}")
        if not IN_COLAB:
            input("\n아무 키나 눌러서 종료하세요...")

if __name__ == "__main__":
    main()



🔗 유튜브 링크 입력
1️⃣ 다운로드할 유튜브 링크를 입력해주세요.
2️⃣ 링크를 하나씩 입력하고 엔터를 누르세요.
3️⃣ 모든 링크를 입력했으면 '다운로드'를 입력하세요.
--------------------------------------------------------------------------------

🔗 링크 #1 입력:
   │ https://youtube.com/shorts/czvWAcpj3ns?si=xMkCnlyabxvEC-FL
✅ 추가됨: 1번째 영상

🔗 링크 #2 입력:
   │ https://youtube.com/shorts/NnpQ9mwoHCQ?si=p0LJOhzoImQVVMeE]
✅ 추가됨: 2번째 영상

🔗 링크 #3 입력:
   │ 다운로드

✅ 총 2개 링크가 입력되었습니다.

🎥 영상 확인 중...
[1] ㅇㅎ?) 수요없는 서비스 씬을 그리는 웹툰작가
[2] 누구나 한번쯤 있었을, 허거덩 모먼트.

🎯 화질 선택
1. 📱 일반화질 (720p 이하, 빠른 다운로드)
2. 🎬 고화질 (최고 품질, 용량 큼)
----------------------------------------
선택하세요 (1 또는 2): 1
✅ 일반화질 모드가 선택되었습니다.
📁 기존 폴더 사용: 영상저장함

🚀 다운로드 시작! (총 2개 영상)
🎬 일반화질 모드로 다운로드합니다.

1/2 다운로드 중...
✅ 1번째 영상 다운로드 완료!

2/2 다운로드 중...
✅ 2번째 영상 다운로드 완료!

📊 다운로드 완료!
✅ 성공: 2개
❌ 실패: 0개

📁 Colab 내 위치: /content/영상저장함

📦 압축 및 다운로드 준비
📦 ZIP 파일 생성 완료: 0727_유튜브다운로드.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🧹 기존 폴더 '영상저장함' 내용을 모두 삭제했습니다.
⬇️ ZIP 파일 자동 다운로드 시작
